In [3]:
import numpy as np
import torch
from torch import nn

# 为什么大模型 LLMs 采用 LN 而不是 BN？

Layer Normalization（LN）在大模型中优于Batch Normalization（BN），主要是因为LN不依赖于批量大小，能够适应小批量或者每次处理单个样本的情况，尤其在NLP任务和Transformer架构中表现更为稳定和高效。而BN在批量较大、且模型设计依赖批次统计量的情况下表现较好，但在小批量甚至推理时容易出现不稳定的情况，因此在大模型中，LN成为更常见的选择。

# Batch Normalization

$LN(x_i) = \alpha * \frac{x_i  - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}} + \beta , \quad\quad\quad BN(x_i) = \alpha * \frac{x_i  - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} + \beta$


<img src="./images/LayerNorm.png" alt="示例图片" width="600">

### 滑动平均的基本思想

滑动平均是计算一个数据序列中某个值的平均值，但它并不仅仅是固定区间内的简单平均，而是给序列中的每个数据点赋予不同的权重，通常越新的数据点权重越大。

$$
EMA_t = \alpha * x_t + (1 - \alpha) * EMA_{t - 1}
$$

在 Batch Normalization (BN) 中，滑动平均用于 更新全局均值和方差，而不是每次都根据当前批次（batch）重新计算。这种方式有助于在训练过程中逐渐稳定模型的行为，从而提高训练的效率。

1. 训练阶段：

在每一个小批次（batch）的训练中，我们计算当前批次的 均值 和 方差，然后通过滑动平均更新全局均值和方差：

$$
running\_mean_t = (1 - \alpha) * running\_mean_{t - 1} + \alpha * x_{mean, t} \\
running\_var_t = (1 - \alpha) * running\_var_{t - 1} + \alpha * x_{var, t}
$$

这里的 running_mean 和 running_var 是全局的均值和方差，它们会逐步逼近训练数据的整体统计量(而不是单次批次的均值和方差)。

2. 推理阶段：

在推理时，我们不再使用当前批次的均值和方差，而是直接使用训练阶段通过滑动平均计算得到的```全局均值``` 和```全局方差```：

$$
\hat{x} = \frac{x - running\_mean}{\sqrt{runnin\_var + \epsilon}}
$$

这样可以保证在推理时，模型的行为与训练时一致，并且避免了因为批量大小过小导致的均值和方差估计不准确的问题。

```较大的动量值（接近1）```：表示对历史均值和方差的权重更大，当前批次的数据对全局均值和方差的影响较小。

```较小的动量值（接近0）```：表示当前批次的数据对全局均值和方差有较大的影响，历史数据的影响较小。

In [8]:
class MyBN:
    def __init__(self, momentum=0.01, eps=1e-5, feat_dim=2):
        '''
        初始化参数值
        param momentum: 动量, 用于计算每个 batch 均值和方差的滑动均值
        param eps: 防止分母为 0
        param feat_dim: 特征维度
        '''
        # 均值和方差的滑动均值
        
        self._running_mean = np.zeros(shape=(feat_dim, ))
        self._running_var = np.ones(shape=(feat_dim, ))
        # 更新 self._running_xxx 时的动量
        # 用于计算每个批次（batch）均值和方差的滑动平均值。
        # 较大的动量值会使得滑动平均更加依赖历史值，较小的动量值则会使得当前批次的统计量对滑动平均有更大影响。
        self._momentum = momentum
        # 防止分母计算为 0
        self._eps = eps
        # 对应 Batch_Norm 中需要更新的 beta 和 gamma, 采用 pytorch 文档中的初始化值
        self._beta = np.zeros(shape=(feat_dim, ))
        self._alpha = np.ones(shape=(feat_dim, ))
        
    def batch_norm(self, x):
        '''
        BN 向传播
        param x: 数据
        return: BN 输出
        '''
        if self.training:
            x_mean = x.mean(axis=0)
            x_var = x.var(axis=0)
            # 对应 running_mean 的更新公式
            self._running_mean = (1 - self._momentum) * x_mean + self._momentum * self._running_mean
            self._running_var = (1 - self._momentum) * x_var + self._momentum * self._running_var
            # 对应论文中计算 BN 的公式
            x_hat = (x - x_mean) / np.sqrt(x_var + self._eps)
        else:
            x_hat = (x - self._running_mean) / np.sqrt(self._running_var + self._eps)
            
        return self._alpha * x_hat + self._beta

### 输出矩阵每个元素的意义：

每个元素 $y_{ij}$ 表示 第 $i$ 个样本的第 $j$ 个特征 经历了 BN 处理后的值。它可以被看作是以下步骤的结果：

**标准化**：减去当前批次的均值（对每个特征独立计算），然后除以当前批次的方差，得到标准化后的值 $\hat{x}$。

**缩放和平移**：对标准化后的值进行缩放（通过乘以 $\alpha$）和偏移（通过加上 $\beta$）。

In [10]:
# 测试代码
np.random.seed(3407)

# 模拟输入数据，假设 batch_size=5，特征维度=5
x_train = np.random.randn(5, 5)  # 随机生成一个5x5的输入数据

# 初始化 Batch Normalization
bn = MyBN(momentum=0.1, eps=1e-5, feat_dim=5)

# 训练模式
bn.training = True
print("Training Mode:")
print("Input:\n", x_train)
output_train = bn.batch_norm(x_train)
print("BN Output (Training):\n", output_train)

# 切换到推理模式
bn.training = False
print("\nInference Mode:")
print("Input:\n", x_train)
output_infer = bn.batch_norm(x_train)
print("BN Output (Inference):\n", output_infer)

Training Mode:
Input:
 [[ 1.34150514 -0.83548739  0.96262807  0.39087082 -0.97108807]
 [-1.49278592  1.52599971 -0.90589414 -0.81963949  0.76510518]
 [-0.59215754  1.20825417 -0.39007968  0.01590583 -0.19420368]
 [-2.03628939  1.84351066 -0.50285955  1.43857724  0.12029993]
 [ 0.47568949 -0.63146271  0.13933391 -1.04265689 -0.92899666]]
BN Output (Training):
 [[ 1.45330954 -1.2936097   1.71077756  0.44181974 -1.11183009]
 [-0.83214406  0.80212113 -1.18996569 -0.91471726  1.53498425]
 [-0.10591506  0.52013393 -0.38920169  0.02162185  0.07252467]
 [-1.27040249  1.08390013 -0.56428414  1.61591344  0.5519832 ]
 [ 0.75515207 -1.11254549  0.43267396 -1.16463777 -1.04766203]]

Inference Mode:
Input:
 [[ 1.34150514 -0.83548739  0.96262807  0.39087082 -0.97108807]
 [-1.49278592  1.52599971 -0.90589414 -0.81963949  0.76510518]
 [-0.59215754  1.20825417 -0.39007968  0.01590583 -0.19420368]
 [-2.03628939  1.84351066 -0.50285955  1.43857724  0.12029993]
 [ 0.47568949 -0.63146271  0.13933391 -1.0426

# Layer Normalization

$LN(x_i) = \alpha * \frac{x_i  - \mu_L}{\sqrt{\sigma_L^2 + \epsilon}} + \beta , \quad\quad\quad BN(x_i) = \alpha * \frac{x_i  - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} + \beta$


<img src="./images/LayerNorm.png" alt="示例图片" width="600">

In [17]:
class LN(nn.Module):
    # 初始化
    
    def __init__(self, normalized_shape, eps: float = 1e-5, elementwise_affine: bool = True):
        super(LN, self).__init__()
        # 需要对哪个维度特征做 LN, torch.size 查看维度
        self.normalized_shape = normalized_shape # [c, w * h]
        self.eps = eps # 防止分母为 0
        self.elementwise_affine = elementwise_affine # affine -仿射, 是否使用可学习的缩放因子和偏移因子
        # 构造可学习的缩放因子和偏置
        if self.elementwise_affine:
            self.gain = nn.Parameter(torch.ones(normalized_shape)) # [c, w * h]
            self.bias = nn.Parameter(torch.zeros(normalized_shape)) # [c, w * h]
        
    # 前向传播
    
    def forward(self, x: torch.Tensor): # [b, c, w * h]
        # 需要做 LN 的维度和输入特征图对应维度的 shape 相同
        assert self.normalized_shape == x.shape[-len(self.normalized_shape):] # [-2:]
        # 需要做 LN 的维度索引
        dims = [-(i + 1) for i in range(len(self.normalized_shape))] # [b, c, w * h] 维度上取 [-1, -2] 维度, 即 [w * h, c]
        # 计算特征图对应维度的均值和方差
        mean = x.mean(dim=dims, keepdim=True) # [b, 1, 1]
        mean_x2 = (x**2).mean(dim=dims, keepdim=True) # [b, 1, 1]
        var = mean_x2 - mean**2 # [b, 1, 1]
        x_norm = (x - mean) / torch.sqrt(var + self.eps) # [b, c, w * h] (mean, var 广播)
        # 线性变换
        if self.elementwise_affine:
            x_norm = self.gain * x_norm + self.bias # [b, c, w * h]
        return x_norm

In [19]:
if __name__ == '__main__':
    x = torch.linspace(0, 23, 24, dtype=torch.float32) # 构造输入层
    x = x.reshape([2, 3, 2*2]) # [b, c, w*h]
    # 实例化
    ln = LN(x.shape[1:])
    # 前向传播
    x = ln(x)
    print(x.shape)
    print(x)

torch.Size([2, 3, 4])
tensor([[[-1.5933, -1.3036, -1.0139, -0.7242],
         [-0.4345, -0.1448,  0.1448,  0.4345],
         [ 0.7242,  1.0139,  1.3036,  1.5933]],

        [[-1.5933, -1.3036, -1.0139, -0.7242],
         [-0.4345, -0.1448,  0.1448,  0.4345],
         [ 0.7242,  1.0139,  1.3036,  1.5933]]], grad_fn=<AddBackward0>)
